In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24993029
paper_name = 'walker_jiranek_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [77]:
original_data = pd.read_excel('raw_data/12864_2013_6243_MOESM1_ESM.xlsx', sheet_name="Add' file 1 BMC Genomics 2013", skiprows=10)

In [78]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 413 x 38


In [79]:
original_data.head()

,Gene name,Systematic Name,Average,±,SD @180 h,Average.1,±.1,SD @ 137 h,Average.2,±.2,...,±.8,SD @ 56 h.1,Unnamed: 30,Unnamed: 31,Unnamed: 32,Gene name.1,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,NaN,BY4743,-0.578632,±,0.0591673,2.48459,±,4.76508,1.56616,±,...,±,0.0608632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PAU8,YAL068C,7.29814,±,0.072242,1.94381,±,2.57873,1.227,±,...,±,0.0219203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CCR4,YAL021C,22.7759,±,0.624761,55.8968,±,2.70858,0.7395,±,...,±,0.0219203,149,99,1.50505,CCR4,YAL021C,NaN,NaN,NaN
3,ADE1,YAR015W,165.411,±,0.782509,225.744,±,17.4048,0.175,±,...,±,0.00424264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,YAR015W,137.931,±,0.868976,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
original_data['orf'] = original_data['Systematic Name'].astype(str)

In [81]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [82]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [83]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                     Gene name  \
index_input                                                      
0                                                          NaN   
66                                                         NaN   
126                                                        NaN   
389                                                        NaN   
390                                                        NaN   
391                                                        NaN   
392                                                        NaN   
393                                       Total deletants: 336   
394                                                        NaN   
395          Genes related in function to identified genes ...   
396                                                        NaN   
397                                                        NaN   
398                                                        NaN   
399       

In [84]:
original_data['data'] = pd.to_numeric(original_data['Average'], errors='coerce')

In [85]:
original_data.loc[original_data['orf']=='BY4743','data']

index_input
0   -0.578632
Name: data, dtype: float64

In [90]:
# Normalize to WT
original_data['data'] = original_data.loc[original_data['orf']=='BY4743','data'].values - original_data['data'].values

In [92]:
original_data = original_data.loc[t,:]

In [93]:
original_data.set_index('orf', inplace=True)

In [94]:
original_data = original_data[['data']].copy()

In [95]:
original_data = original_data.groupby(original_data.index).mean()

In [96]:
original_data.shape

(348, 1)

# Prepare the final dataset

In [97]:
data = original_data.copy()

In [98]:
dataset_ids = [11811]
datasets = datasets.reindex(index=dataset_ids)

In [99]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [100]:
data.head()

dataset_id,11811
data_type,value
orf,
YAL021C,-23.354524
YAL047C,-9.927619
YAL068C,-7.876768
YAR015W,-152.249731
YAR018C,-7.718761


## Subset to the genes currently in SGD

In [101]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 3


In [102]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,11811
,data_type,value
gene_id,orf,
19,YAL021C,-23.354524
45,YAL047C,-9.927619
1869,YAL068C,-7.876768
67,YAR015W,-152.249731
68,YAR018C,-7.718761


# Normalize

In [103]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [104]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [105]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            11811          
data_type             value    valuez
gene_id orf                          
19      YAL021C  -23.354524 -1.458617
45      YAL047C   -9.927619 -0.620034
1869    YAL068C   -7.876768 -0.491947
67      YAR015W -152.249731 -9.508825
68      YAR018C   -7.718761 -0.482079

# Print out

In [106]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [107]:
from IO.save_data_to_db3 import *

In [108]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24993029...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.30it/s]

Updating the data_modified_on field...
